Checking the required libraries and installing it if doesn't exist

In [ ]:
!pip install pyodbc

Import all relevant libraries for use 

In [1]:
import pandas as pd
import os
import pyodbc
import numpy as np

pyodbc.pooling = False

Separating files in their order of execution

In [2]:
path = r'C:\OlistPrimaryDataSet\\'

olist_source_files = os.listdir(path)
base_tbls = []
intermediate_tbls = []
dependent_tbls = []

for file in olist_source_files:
    file_name = (file.split(".")[0]).upper()
    if file_name in ['OLIST_PRODUCTS','OLIST_SELLERS', 'OLIST_CUSTOMERS', 'OLIST_GEOLOCATION', 'PRODUCT_CATEGORY_NAME_TRANSLATION']:
        base_tbls.append(file)
    elif file_name in ['OLIST_ORDERS']:
        intermediate_tbls.append(file)
    else:
        dependent_tbls.append(file)

In [3]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [4]:
def construct_query(table_name, columns, number_of_columns):
    ''' Constructs a SQL INSERT query.
    
        Args:
            table_name: Name of the table to be inserted.
            columns: A sequence of strings representing the column name in database.
            number_of_columns: Numbers of columns in the table

        Returns:
            A string which represents a SQL INSERT query. For example:

            INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
            VALUES (?, ?, ?, ?, ?, ?);'''
    if table_name and columns and number_of_columns:
        sql = '''INSERT INTO ''' + table_name + ''' (''' + columns + ''') VALUES (''' + '''?,'''*(number_of_columns-1) + '''?)'''
    return sql

In [20]:
def load_dataframes(file):
    '''Takes a csv file, converts it to Pandas Dataframe and Loads it into a Table in SQL Server.
    
        Args: Takes a file as an argument.
    '''
    path = r'C:\OlistPrimaryDataSet\\'

    table_name = (file.split(".")[0]).upper()
    print("Inserting data into " + table_name + " Table")

    dataframe = pd.read_csv(path+file)
    dataframe = dataframe.drop_duplicates()
    dataframe = dataframe.astype(str)

    # creating column list for insertion
    cols = ",".join([str(i) for i in dataframe.columns.tolist()])
    query = construct_query(table_name, cols, len(cols.split(",")))

    #Inserting data into table in chunks
    for subset in chunker(dataframe.values.tolist(), 1000):
        cursor.execute(query, tuple(row))
    
    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

    

Establishing a DB Connection an creating Tables for Data Ingestion

In [23]:
connection_str = 'Driver={SQL Server};' + 'Server=DESKTOP-K4UDQKF;' + 'Database=OLISTSTORE_DB;' + 'Trusted_Connection=yes;'
connection = pyodbc.connect(connection_str)

cursor = connection.cursor()
cursor.fast_executemany = True

Loading data into the tables in the order of dependency

In [24]:
order = base_tbls+intermediate_tbls+dependent_tbls

for item in order:
    load_dataframes(item)

cursor.close()
connection.close()

Inserting data into OLIST_ORDER_REVIEWS Table
